In [1]:
import os 
BASE_DIR=os.path.abspath("../../")
import sys 
sys.path.append(BASE_DIR)
DATA_DIR="/home/arturo/Documents/programacion_stuff/DeepElasticity/data/002-Neo"
from src.utils.data import DataNeo
from src.models import PINN  
from src.train import Trainer
import torch 
import matplotlib.pyplot as plt 
import time
from datetime import datetime
import pickle
import numpy as np


In [2]:
data=DataNeo("ARTURO_TEST_2_NEOH",6,E="E10kPa",base_dir=DATA_DIR)
data.load_data(load_stage=6)
train_init_pos_main,train_disp_main,test_init_pos_main,test_disp_main,position_selected_stresses,return_stress,return_colloc_points=data.prepare_pytorch_data()


In [8]:
init_values={"nu":.35,"E":float(0.001)}
loss_weights_init={"data":1e1,"PDE":1e2,"BC":1e6}

pinn=PINN([3,50,50,50,50,3],use_of_alpha=False,train_E=True,init_values=init_values,loss_weights_init=loss_weights_init)
step_dict = {
    "step_1": {"optim": torch.optim.Adam(pinn.parameters(), lr=1e-2), 
            "epochs": 15},
    "step_2": {"optim": torch.optim.Adam(pinn.parameters(), lr=1e-3), 
            "epochs": 15},
    "step_3": {"optim": torch.optim.Adam(pinn.parameters(), lr=1e-4), 
            "epochs": 15}
}

In [9]:
trainer=Trainer(step_dict)
trainer.train(pinn,data)

step_1


/home/arturo/miniconda3/envs/pinns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1339, 5])) that is different to the input size (torch.Size([1339, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (6) must match the size of tensor b (5) at non-singleton dimension 1